# Mid-term Project : Genetic Variations of 2019-nCoV
## Index
<ol>
     <li> Preparation
     <li> The Propagation Path
     <li> The Mutation Rate
     <li> Parts with Higher Rate of Mutations

## 1. Preparation
<ol>
     <li> Get a funtion for read fasta files
     <li> Get a funtion to write fasta files
     <li> Get sequences of proteins of the reference genome.
     <li> Clean the sequences of all genome file.
     <li> translate the cleaned genome file by 6 frame using transeq
     <li> align the reference protein sequence and the translated file using phmmer

In [5]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [6]:
# The function to write sequences to a FASTA file
def write_fasta(tmp_filename, genome_list):
    f = open(tmp_filename, 'w')
    for genome in list(genome_list.keys()):
        if len(genome_list[genome]) == 1:
            f.write(">{}\n".format(genome))
            f.write("{}\n".format(genome_list[genome][0]))
        else:
            for idx in range(len(genome_list[genome])):
                f.write(">{} #{}\n".format(genome, idx+1))
                f.write("{}\n".format(genome_list[genome][idx]))
    f.close()

In [7]:
# Get the reference protein sequence fasta file from ../data/GCF_009858895.2_ASM985889v3_protein.faa
filename_refprot = '../data/GCF_009858895.2_ASM985889v3_protein.faa'
ref_protein = read_fasta(filename_refprot)

for prot in ref_protein.keys():
    tempdict = {str(prot[15:-39]) : [ref_protein[prot]] }                                 
    write_fasta("REF_{}.fa".format("".join(str(prot[15:-39]).replace(' ', ''))), tempdict)

In [8]:
# Clean again 2019nCoV_genomes.2020_04_14.clean.fa ;
# 1) Extract only USA sequences and the reference 2) Exclude sequneces with too much 'N's. I aribtraily set 'N'*100.

filename_0414_cleaned_raw = '2019nCoV_genomes.2020_04_14.clean.fa'
genome_0414_raw = read_fasta(filename_0414_cleaned_raw)
genome_0414 = {genome.replace('|', '@') : [genome_0414_raw[genome]] for genome in genome_0414_raw.keys() if 'USA' in genome or 'NC_045512' in genome}

list_to_delete = []
for name, sequence in genome_0414.items():
    if 'N'*100 in sequence[0]:
        list_to_delete.append(name)
for name in list_to_delete:
    del genome_0414[name]
write_fasta('my_0414_genome_cleaned.fa', genome_0414)


In [9]:
# I commanded in Ubuntu;
# 1) Transaltion of my_0414_genome_cleaned.fa using transeq
# 2) sequnece alignments using phmmer
# E, M, N, S, RdRP, orf1ab, orf10, orf3a, orf6, orf7a, orf7b, orf8, orf10.

translated_genome_0414 = read_fasta('my_0414_prot_trans6')


## 2. The Propagation Path
<ol>
     <li> Get a function to read the phmmerred result and get an aligned sequence.
     <li> Merge the aligned sequences of the genomes by all proteins, to draw a phylogenic tree at one time.
     <li> Exclude overlaps and sequences with deletion before export the fasta file to Megasoftware.
     <li> Keep analyzing at Megasoftware.

In [10]:
# A function to read phmmered result and extract a corresponding aligned sequence from the cleaned genome file.
# Using write_fasta function and this function at the same time, I can get a fasta file of the extracted sequences.
def read_phmmer_get_sequence(filename_phmmerresult):
    aligned_sequences = dict()
    
    # get the index
    f = open(filename_phmmerresult, 'r')
    for line in f.readlines():
        tokens = line.strip().split()
        if not tokens[0].startswith('#'):
            ref_idx_srt, ref_idx_end = int(tokens[15]), int(tokens[16])
            break
    f.close()
    
    # if it aligned perfectly, get the sequence from the sequence file. 
    f = open(filename_phmmerresult, 'r')
    for line in f.readlines():
        tokens = line.strip().split()
        if not tokens[0].startswith('#') and tokens[0][-2] not in aligned_sequences.keys():
            query_idx_srt, query_idx_end = int(tokens[15]), int(tokens[16])
            this_idx_srt,   this_idx_end = int(tokens[17]), int(tokens[18])
            if (ref_idx_srt, ref_idx_end) == (query_idx_srt, query_idx_end):
                aligned_sequences[tokens[0][:-2]] = [translated_genome_0414[tokens[0]][this_idx_srt-1 : this_idx_end]]
            # This elif case is only for ORF1ab. 
            # 89 is the optimal number to consider too much DNA deletion in ORF1ab in some sequneces.
            elif ref_idx_srt > query_idx_srt:
                this_idx_srt = 89
                this_idx_end = ref_idx_end
                aligned_sequences[tokens[0][:-2]] = [translated_genome_0414[tokens[0]][this_idx_srt-1 : this_idx_end]]
            else:
                this_idx_srt -= (query_idx_srt - ref_idx_srt)
                this_idx_end += (ref_idx_end - query_idx_end)
                aligned_sequences[tokens[0][:-2]] = [translated_genome_0414[tokens[0]][this_idx_srt-1 : this_idx_end]]
    f.close()
    return aligned_sequences

In [11]:
# This block is to run the funtions above and assign the result in the code in the dictionary data type.

write_fasta('alignedresult_E.fa', read_phmmer_get_sequence('phmmerresult_E_protein.fa'))
aligned_E = read_fasta('alignedresult_E.fa')
write_fasta('alignedresult_M.fa', read_phmmer_get_sequence('phmmerresult_M_protein.fa'))
aligned_M = read_fasta('alignedresult_M.fa')
write_fasta('alignedresult_N.fa', read_phmmer_get_sequence('phmmerresult_N_protein.fa'))
aligned_N = read_fasta('alignedresult_N.fa')
write_fasta('alignedresult_S.fa', read_phmmer_get_sequence('phmmerresult_S_protein.fa'))
aligned_S = read_fasta('alignedresult_S.fa')
write_fasta('alignedresult_RdRP.fa', read_phmmer_get_sequence('phmmerresult_RdRP.fa'))
aligned_RdRP = read_fasta('alignedresult_RdRP.fa')
write_fasta('alignedresult_orf1ab.fa', read_phmmer_get_sequence('phmmerresult_orf1ab_protein.fa'))
aligned_orf1ab = read_fasta('alignedresult_orf1ab.fa')
write_fasta('alignedresult_ORF3a.fa', read_phmmer_get_sequence('phmmerresult_ORF3a_protein.fa'))
aligned_orf3a = read_fasta('alignedresult_ORF3a.fa')
write_fasta('alignedresult_ORF6.fa', read_phmmer_get_sequence('phmmerresult_ORF6_protein.fa'))
aligned_orf6 = read_fasta('alignedresult_ORF6.fa')
write_fasta('alignedresult_ORF7a.fa', read_phmmer_get_sequence('phmmerresult_ORF7a_protein.fa'))
aligned_orf7a = read_fasta('alignedresult_ORF7a.fa')
write_fasta('alignedresult_ORF7b.fa', read_phmmer_get_sequence('phmmerresult_ORF7b_protein.fa'))
aligned_orf7b = read_fasta('alignedresult_ORF7b.fa')
write_fasta('alignedresult_ORF8.fa', read_phmmer_get_sequence('phmmerresult_ORF8_protein.fa'))
aligned_orf8 = read_fasta('alignedresult_ORF8.fa')
write_fasta('alignedresult_ORF10.fa', read_phmmer_get_sequence('phmmerresult_orf10_protein.fa'))
aligned_orf10 = read_fasta('alignedresult_ORF10.fa')

In [12]:
# Murge the aligned sequences to draw one phylogenic tree that considers all the results.
# Get the translated sequence of only protein coded site.
# Because megasoftware only recieves a fasta file with the only identical number of sequences,
# genomes that have different number of sequences with the reference are excluded (total 3 genomes).

for x,y in aligned_S.items():
    if len(y) != 1273:
        print(x, len(y))
        
for x,y in aligned_orf3a.items():
    if len(y) != 275:
        print(x, len(y))

aligned_and_merged = {genome.replace('@', '_') : [aligned_S[genome]+aligned_orf3a[genome]+aligned_E[genome]+aligned_M[genome]+aligned_orf6[genome]+aligned_orf7a[genome]+aligned_orf7b[genome]+aligned_orf8[genome]+aligned_N[genome]+aligned_RdRP[genome]+aligned_orf10[genome]] for genome in genome_0414.keys() if genome not in ['MT263466@USA_WA@2020-03-23','MT326059@USA@2020-03-24', 'MT293186@USA_WA@2020-03-26']}
write_fasta('aligned_and_merged.fa', aligned_and_merged)

MT263466@USA_WA@2020-03-23 1272
MT326059@USA@2020-03-24 274
MT293186@USA_WA@2020-03-26 272


In [13]:
# To exclude overlaps. / Identical with what I used in Assignment 1.


identical_sequences = []
name_of_sequences = list(aligned_and_merged.keys())
sequences_to_check = list(aligned_and_merged.values())

#erase a sequence from the list and see if the list has the same ones. 
while(len(sequences_to_check) > 1):
    current_sequence = sequences_to_check.pop(0)
    if current_sequence not in sequences_to_check:
        name_of_sequences.pop(0)
    else:
        templist = []
        templist.append(name_of_sequences.pop(0).split()[0])
        while current_sequence in sequences_to_check:
            idx = sequences_to_check.index(current_sequence)
            sequences_to_check.pop(idx)
            templist.append(name_of_sequences.pop(idx).split()[0])
        identical_sequences.append(templist)
        

for sequences in identical_sequences:
    print(sorted(sequences), "are identical with each other.")
print("number of identical sequences: {}".format(len(identical_sequences)))

aligned_and_merged_wo_ovlps = aligned_and_merged.copy()
for i in identical_sequences:
    i.sort()
for idx1 in range(len(identical_sequences)):
    for idx2 in range(len(identical_sequences[idx1])-1): 
        aligned_and_merged_wo_ovlps.pop(identical_sequences[idx1][idx2])
        
        
write_fasta('file_to_mega.fa', aligned_and_merged_wo_ovlps)


['MT044258_USA_CA_2020-01-27', 'MT106053_USA_CA_2020-02-10', 'MT118835_USA_CA_2020-02-23', 'MT159705_USA_2020-02-17', 'MT159706_USA_2020-02-17', 'MT159707_USA_2020-02-17', 'MT159708_USA_2020-02-17', 'MT159709_USA_2020-02-20', 'MT159710_USA_2020-02-17', 'MT159711_USA_2020-02-20', 'MT159712_USA_2020-02-25', 'MT159713_USA_2020-02-18', 'MT159714_USA_2020-02-18', 'MT159715_USA_2020-02-24', 'MT159717_USA_2020-02-17', 'MT159719_USA_2020-02-18', 'MT159720_USA_2020-02-21', 'MT159721_USA_2020-02-21', 'MT159722_USA_2020-02-21', 'MT184907_USA_2020-02-18', 'MT184909_USA_2020-02-21', 'MT184912_USA_2020-02-17', 'MT188340_USA_MN_2020-03-07', 'MT276331_USA_TX_2020-02-29', 'MT304482_USA_IL_2020-03-01', 'MT326035_USA_2020-03-23', 'NC_045512_China_2019-12-00'] are identical with each other.
['MN985325_USA_2020-01-19', 'MN997409_USA_AZ_2020-01-22', 'MT020880_USA_WA_2020-01-25', 'MT020881_USA_WA_2020-01-25', 'MT163717_USA_WA_2020-02-28', 'MT163718_USA_WA_2020-02-29', 'MT163719_USA_WA_2020-03-01', 'MT188339_

### This part is continued by using MEGA software. 

## II. The Mutation Rate
<ol>
     <li> Express the date infomation in julian date.
     <li> Align the reference protein with the merged protein sequence in Part2, by using blastp.
     <li> Read the blastp result, store the number of mutations by dates and weeks

In [14]:
# Exclude data without the exact date info.
# Julian date is an year-based date that have 1 Jan as 1 and 31 Dec as 365.


Reference = dict()
Reference['NC_045512_China_2019-12-00'] = aligned_and_merged['NC_045512_China_2019-12-00']
write_fasta('REF_for2and3.fa', Reference)


genome_0414 = {genome : {'seq' : aligned_and_merged[genome]} for genome in aligned_and_merged.keys()}
for info in genome_0414:
    if (info.split('_')[-1]).split('-')[-1] == '00' or (info.split('_')[-1]).split('-')[-1] == '2020':
        continue
    genome_0414[info]['date'] = info.split('_')[-1]
    month = genome_0414[info]['date'].split('-')[1]
    day = int(genome_0414[info]['date'].split('-')[2])
    if month == '01':
        genome_0414[info]['julian'] = day
    elif month == '02':    
        genome_0414[info]['julian'] = day + 31
    elif month == '03':    
        genome_0414[info]['julian'] = day + 60
    elif month == '04':    
        genome_0414[info]['julian'] = day + 91

In [15]:
# Did blastp for query 'NC_045512_China_2019-12-00' in aligned_and_merged and db for all genome in aligned_and_merged;
# Got 'blastpresult_q2.re'.
# Get daily and weekly based average number of mutations(# of mismathes + gaps)

# Collect number of mutations by sequences from the blastp result.
f = open('blastpresult_q2.re', 'r')
for line in f.readlines():
    if not line.startswith('#'):
        genome_0414[line.split()[1]]['total_muts'] = int(line.split()[4]) + int(line.split()[5])
f.close()

# Get daily based data.
muts_by_date = dict()
for info in genome_0414.keys():
    if 'julian' in genome_0414[info] and 'total_muts' in genome_0414[info]:
        idx2+= 1
        if genome_0414[info]['julian'] not in muts_by_date.keys():
            muts_by_date[genome_0414[info]['julian']] = [genome_0414[info]['total_muts']]
        else:
            muts_by_date[genome_0414[info]['julian']].append(genome_0414[info]['total_muts'])
avg_muts_by_date = {julian : (sum(muts_by_date[julian])/len(muts_by_date[julian])) for julian in muts_by_date.keys()}

# Get and print weekly based data.
muts_by_week = dict()
muts_by_week.clear()
for julian in avg_muts_by_date.keys():
    if 'week{}'.format(int(int(julian/7) + 1)) not in muts_by_week.keys():
        muts_by_week['week{}'.format(int(julian/7) + 1)] = [avg_muts_by_date[julian]]
    elif 'week{}'.format(int(int(julian/7) + 1)) in muts_by_week.keys():
        muts_by_week['week{}'.format(int(julian/7) + 1)] += [avg_muts_by_date[julian]]
avg_muts_by_week = {int(week[4:]) : (sum(muts_by_week[week])/len(muts_by_week[week])) for week in muts_by_week.keys()}
for week in sorted(avg_muts_by_week):
    print('week', week, ':', avg_muts_by_week[week])

week 3 : 1.0
week 4 : 1.4
week 5 : 1.5
week 6 : 1.0
week 7 : 1.125
week 8 : 0.30666666666666664
week 9 : 1.9583333333333335
week 10 : 2.527934250623326
week 11 : 3.4592883307169022
week 12 : 5.6428010527678305
week 13 : 6.545434960377281
week 14 : 3.0


### For conclusion, I could check that from week8, which is the week that first American was infected(19 Mar), almost one protein mutation per one week is occured.

 
  
   
    
     
     


## III. Parts with Higher Rate of Mutation
<ol>
     <li> align the reference protein one by one ib this time to the merged protein sequences in Part 2, using blastp.
     <li> Get a function to read blast result file and store the number of mutations in order of protein names.
     <li> Print and compare the result.

In [16]:
# Did multiple blastp, query for all the reference sequences and db for fasta file of the dictionary 'aligned_and_merged'.

In [32]:
# The function to get the number of mismatches from the blast result file.
# RdRP is not properly sequenced; 
# Even the reference protein shows to have 6 mismatches with RdRP sequence I used.

def read_blast_get_mutation(blastresult_filename, protein_name, genome_0414):
    num_of_muts = []
    f = open(blastresult_filename, 'r')
    for line in f.readlines():
        if not line.startswith('#'):
            num_of_muts.append(int(line.split()[4]) + int(line.split()[5]))
    if protein_name != 'RdRP':
        print("The average number of the mutation in {} protein is ".format(protein_name),sum(num_of_muts)/len(num_of_muts))
    elif protein_name == 'RdRP':
        print("The average number of the mutation in {} protein is ".format(protein_name),sum(num_of_muts)/len(num_of_muts)-6)

In [33]:
read_blast_get_mutation('blastresult_S.re', 'S', genome_0414)
read_blast_get_mutation('blastresult_orf8.re', 'orf8', genome_0414)
read_blast_get_mutation('blastresult_RdRP.re', 'RdRP', genome_0414)
read_blast_get_mutation('blastresult_orf3a.re', 'orf3a', genome_0414)
read_blast_get_mutation('blastresult_M.re', 'M', genome_0414)
read_blast_get_mutation('blastresult_N.re', 'N', genome_0414)
read_blast_get_mutation('blastresult_orf6.re', 'orf6', genome_0414)
read_blast_get_mutation('blastresult_E.re', 'E', genome_0414)
read_blast_get_mutation('blastresult_orf7a.re', 'orf7a', genome_0414)
read_blast_get_mutation('blastresult_orf7b.re', 'orf7b', genome_0414)
read_blast_get_mutation('blastresult_orf10.re', 'orf10', genome_0414)

The average number of the mutation in S protein is  0.7766666666666666
The average number of the mutation in orf8 protein is  0.53
The average number of the mutation in RdRP protein is  0.49666666666666703
The average number of the mutation in orf3a protein is  0.475
The average number of the mutation in M protein is  0.14666666666666667
The average number of the mutation in N protein is  0.12166666666666667
The average number of the mutation in orf6 protein is  0.005
The average number of the mutation in E protein is  0.005
The average number of the mutation in orf7a protein is  0.011666666666666667
The average number of the mutation in orf7b protein is  0.0
The average number of the mutation in orf10 protein is  0.0


In [34]:
# The function to get the number of mismatches from the blast result file.

def read_blast_get_late_mutation(blastresult_filename, protein_name, genome_0414):
    num_of_muts = []
    f = open(blastresult_filename, 'r')
    for line in f.readlines():
        if not line.startswith('#') and 'julian' in genome_0414[line.split()[1]].keys() and genome_0414[line.split()[1]]['julian'] >84 :
            num_of_muts.append(int(line.split()[4]) + int(line.split()[5]))
    if protein_name != 'RdRP':
        print("The average number of the mutation in {} protein is ".format(protein_name),sum(num_of_muts)/len(num_of_muts))
    elif protein_name == 'RdRP':
        print("The average number of the mutation in {} protein is ".format(protein_name),sum(num_of_muts)/len(num_of_muts)-6)

In [35]:
read_blast_get_late_mutation('blastresult_S.re', 'S', genome_0414)
read_blast_get_late_mutation('blastresult_orf8.re', 'orf8', genome_0414)
read_blast_get_late_mutation('blastresult_RdRP.re', 'RdRP', genome_0414)
read_blast_get_late_mutation('blastresult_orf3a.re', 'orf3a', genome_0414)
read_blast_get_late_mutation('blastresult_M.re', 'M', genome_0414)
read_blast_get_late_mutation('blastresult_N.re', 'N', genome_0414)
read_blast_get_late_mutation('blastresult_orf6.re', 'orf6', genome_0414)
read_blast_get_late_mutation('blastresult_E.re', 'E', genome_0414)
read_blast_get_late_mutation('blastresult_orf7a.re', 'orf7a', genome_0414)
read_blast_get_late_mutation('blastresult_orf7b.re', 'orf7b', genome_0414)
read_blast_get_late_mutation('blastresult_orf10.re', 'orf10', genome_0414)

The average number of the mutation in S protein is  1.0147058823529411
The average number of the mutation in orf8 protein is  0.5797101449275363
The average number of the mutation in RdRP protein is  0.5362318840579707
The average number of the mutation in orf3a protein is  0.45588235294117646
The average number of the mutation in M protein is  0.07352941176470588
The average number of the mutation in N protein is  0.14492753623188406
The average number of the mutation in orf6 protein is  0.0
The average number of the mutation in E protein is  0.014705882352941176
The average number of the mutation in orf7a protein is  0.014705882352941176
The average number of the mutation in orf7b protein is  0.0
The average number of the mutation in orf10 protein is  0.0


### For conclusion, I could check that the mutation rate was descending in order of S, orf8, RdRP, orf3a, M, N, orf6, E, orf7a, orf7b and orf10 proeins, with one mutation in S protein is ensured in late weeks.
### Also, there was no mutation in orf7b and orf10 in both early and late periods, which can mean that the virus does not prefer to have a mutation in that area.
### Thank you for reading.
